In [72]:
import lzma
s = pickle.dumps(psi_ts_trajs)

In [78]:
s_array = pickle.dumps(np.asarray(psi_ts_trajs, dtype=object))

In [76]:
%%time
s_compressed = lzma.compress(s)

CPU times: user 22min 3s, sys: 5.89 s, total: 22min 9s
Wall time: 22min 24s


In [80]:
sys.getsizeof(s) / (1<<30), \
sys.getsizeof(s_array) / (1<<30), \
sys.getsizeof(s_compressed) / (1<<30), \

(3.680209208279848, 3.6816780203953385, 2.4485338935628533)

# Saving simulation data

To store data simulated by `qutip`, I used SQL tables to store the data. Types like integers, real numbers and strings can be stored easily. However, storing Python objects is not straightforward. In the `qutip` simulations, I used `pickle.dumps` to serialize objects of type `qutip.qobj.Qobj` to `bytes`, which can be stored as a blob in an SQL table.

In [20]:
import itertools
import pickle
import pandas as pd

import sys

In [21]:
columnsHamiltonians = ('hamiltonianUUID', \
            'modelName', \
            'geometry', \
            'xxInteraction', \
            'yyInteraction', \
            'zzInteraction', \
            'xMagneticField', \
            'yMagneticField', \
            'zMagneticField',)

dataHamiltonians = [(None, 'TransverseFieldIsing', '1d-nn', 0.0, 0.0, j_int, bx, 0.0, 0.0)]

dfHamiltonians = pd.DataFrame(data=dataHamiltonians, columns=columnsHamiltonians)
dfHamiltonians

,hamiltonianUUID,modelName,geometry,xxInteraction,yyInteraction,zzInteraction,xMagneticField,yMagneticField,zMagneticField
0,None,TransverseFieldIsing,1d-nn,0.0,0.0,1.0,1.0,0.0,0.0


In [22]:
columnsDecoherence = ('decoherenceUUID', \
            'type', \
            'GammaMinus', \
            'GammaPlus', \
            'GammaZ',)

dataDecoherence = [(None, 'local', Gammam, Gammap, 0.0)]

dfDecoherence = pd.DataFrame(data=dataDecoherence, columns=columnsDecoherence)
dfDecoherence

,decoherenceUUID,type,GammaMinus,GammaPlus,GammaZ
0,None,local,0.01,0.01,0.0


In [23]:
productStates = ('productStateUUID', \
                 'polarAngle', \
                'azimuthalAngle', \
                )

In [24]:
%%time
# pandas.DataFrame cannot save BLOBs to SQL
columnsSimulationResults = ('simulationUUID', 'time', 'trajectoryIndex', 'stateBytes')

columnTypes = ('text', 'real', 'integer', 'blob')

dataSimulationResults = []
dataSimulationResultsCompressed = []

for timeIndex, trajectoryIndex in itertools.product(range(n_steps), range(n_trajectories)):
    state = psi_ts_trajs[trajectoryIndex][timeIndex]
    stateBytes = pickle.dumps(state)
   
    row = (string_uuid.hex, float(ts[timeIndex]), trajectoryIndex, stateBytes)
    
    dataSimulationResults += [(row)]    

CPU times: user 8.93 s, sys: 270 ms, total: 9.2 s
Wall time: 9.24 s


In [25]:
%%time
# pandas.DataFrame cannot save BLOBs to SQL
columnsSimulationResultsCompressed = ('simulationUUID', 'time', 'trajectoryIndex', 'stateBytesCompressed')

columnTypes = ('text', 'real', 'integer', 'blob')

dataSimulationResultsCompressed = []

for timeIndex, trajectoryIndex in itertools.product(range(n_steps), range(n_trajectories)):
    state = psi_ts_trajs[trajectoryIndex][timeIndex]
    stateBytes = pickle.dumps(state)
    stateBytesCompressed = lzma.compress(stateBytes)

    rowCompressed = (string_uuid.hex, float(ts[timeIndex]), trajectoryIndex, stateBytesCompressed)
    
    dataSimulationResultsCompressed +=[(row)]
    

NameError: name 'lzma' is not defined

In [26]:
del stateCompressed

NameError: name 'stateCompressed' is not defined

In [27]:
', '.join([' '.join(c) for c in zip(columns, columnTypes)]), \
', '.join(['?' for _ in range(len(columns))])

NameError: name 'columns' is not defined

In [28]:
%%time
import lzma
import sys

stateCompressed = lzma.compress(stateBytes)

print('sizeof state object = %g MiB; sizeof pickled state object = %g MiB; sizeof compressed picked state object = %g MiB' % \
    (sys.getsizeof(state) / (1<<20), sys.getsizeof(stateBytes) / (1<<20), sys.getsizeof(stateCompressed)/(1<<20)))

sizeof state object = 6.10352e-05 MiB; sizeof pickled state object = 0.00426674 MiB; sizeof compressed picked state object = 0.00142384 MiB
CPU times: user 3.66 ms, sys: 7.04 ms, total: 10.7 ms
Wall time: 11.7 ms


In [29]:
%%time
import os
import pickle

save_dict = {\
    'dfHamiltonians': dfHamiltonians, \
    'dfDecoherence': dfDecoherence, \
    'dfSimulations': dfSimulations
}

outputfile = open(os.path.join('WorkingData', string_uuid.hex + "_mcwf.pkl"), "wb")
pickle.dump(save_dict, outputfile)

NameError: name 'dfSimulations' is not defined

In [30]:
%%time
import sqlite3
conn = sqlite3.connect('WorkingData/%s-compressed.db' % string_uuid.hex)

sqlCreateTable = '''
    CREATE TABLE IF NOT EXISTS
    SimulationResults
    (''' + ', '.join([' '.join(c) for c in zip(columnsSimulationResultsCompressed, columnTypes)]) + ')'

sqlInsert = '''
    INSERT INTO SimulationResults
''' + str(columnsSimulationResultsCompressed) + 'VALUES (' + ', '.join(['?' for _ in range(len(columnsSimulationResultsCompressed))]) + ')'


dfHamiltonians.to_sql('Hamiltonians', con=conn, if_exists='append')
dfDecoherence.to_sql('Decoherence', con=conn, if_exists='append')

conn.execute(sqlCreateTable)
for row in dataSimulationResultsCompressed:
    #print(row)
    conn.execute(sqlInsert, row)

conn.commit()
conn.close()

OperationalError: unable to open database file

In [31]:
conn = sqlite3.connect('WorkingData/%s.db' % string_uuid.hex)
cursor = conn.execute('''
    SELECT COUNT(*) FROM SimulationResults
''')
print(cursor.fetchall())
conn.close()

OperationalError: unable to open database file

In [32]:
sizePickle = 2535363205 
sizeSqlite3 = 2540417024

(sizePickle - sizeSqlite3) / (1<<20)

-4.819697380065918